# Saarthi TTS Training

In [6]:
import os
from glob import glob
from trainer import Trainer, TrainerArgs
import torch
from TTS.tts.configs.shared_configs import BaseDatasetConfig
from TTS.tts.configs.vits_config import VitsConfig
from TTS.tts.datasets import load_tts_samples
from TTS.tts.models.vits import CharactersConfig, Vits, VitsArgs, VitsAudioConfig
from TTS.tts.utils.languages import LanguageManager
from TTS.tts.utils.speakers import SpeakerManager
from TTS.tts.utils.text.tokenizer import TTSTokenizer
from TTS.utils.audio import AudioProcessor
from TTS.config import load_config
from TTS.tts.models import setup_model

In [7]:
output_path = './'

mailabs_path = "hindi_temp/**"
dataset_paths = glob(mailabs_path)
dataset_config = [
    BaseDatasetConfig(name="mailabs", meta_file_train=None, path=path, language=path.split("/")[-1])
    for path in dataset_paths
]

In [8]:
phonems = '|k_|K_|g_|j_|J_|X_|x_f_K_k_G_g_|y_w_h_C_c_J_i_y_Y_V_q_x_X_N_S_r_T_t_D_d_n~_n_`I_I_P_p_B_b_m_z_s_Aː_A_Iː_i_u_Uː_R_Oː_O_eː_e_Eː_oː_o_M_h_a_`a_v_j_l_`l'
dev = ''.join([chr(i) for i in range(0x0900, 0x097F)])

In [9]:
audio_config = VitsAudioConfig(
    sample_rate=16000,
    win_length=1024,
    hop_length=256,
    num_mels=80,
    mel_fmin=0,
    mel_fmax=None,
)

vitsArgs = VitsArgs(
    use_speaker_embedding=True,
    speaker_embedding_channels=512,
    use_language_embedding=True,
    embedded_language_dim=512,
    use_sdp=False,
    num_speakers=50,
    # freeze_encoder=True,
    # freeze_PE=True,
    # freeze_DP=True,
    # freeze_flow_decoder=True,
    # freeze_waveform_decoder=True

)

In [16]:
config = VitsConfig(
    model_args=vitsArgs,
    audio=audio_config,
    speaker_embedding_channels=512,
    run_name="hindi_female",
    batch_size=48,
    eval_batch_size=16,
    batch_group_size=0,
    num_loader_workers=4,
    num_eval_loader_workers=4,
    run_eval=True,
    test_delay_epochs=-1,
    epochs=1000,
    save_step=100,
    save_n_checkpoints=10,
    text_cleaner=None,
    use_phonemes=False,
    phoneme_language="en-us",
    phoneme_cache_path=os.path.join(output_path, "phoneme_cache"),
    compute_input_seq_cache=False,
    use_language_weighted_sampler=False,
    print_eval=False,
    mixed_precision=False,
    # sort_by_audio_len=True,
    min_audio_len=16 * 128 * 4,
    max_audio_len=160000,
    output_path=output_path,
    datasets=dataset_config,
    characters=CharactersConfig(
        pad="<PAD>",
        eos="<EOS>",
        bos="<BOS>",
        blank="<BLNK>",
        characters= dev,
        punctuations= ",?.!;:'‘¡",
        phonemes= dev,
        
    ),
    test_sentences=[
        [
            "हाय क्या मैं ओंकार से बात कर रहा हूँ ?",
            "hindi_female",
            None,
            "hindi",
        ],
    ],
    num_speakers=50,
    use_speaker_embedding = True,

)

In [17]:
config.from_dict(config.to_dict())

# init audio processor
ap = AudioProcessor(**config.audio.to_dict())

# load training samples
train_samples, eval_samples = load_tts_samples(
    dataset_config,
    eval_split=True,
    eval_split_max_size=config.eval_split_max_size,
    eval_split_size=config.eval_split_size,
)

# print(eval_samples)
speaker_manager = SpeakerManager()
# speaker_manager.load_ids_from_file('model_config/speakers.pth')
# speaker_manager = SpeakerManager(speaker_id_file_path='model_config/speakers.pth')
speaker_manager.set_ids_from_data(train_samples + eval_samples, parse_key="speaker_name")
config.model_args.num_speakers = 50

 > Setting up Audio Processor...
 | > sample_rate:16000
 | > resample:False
 | > num_mels:80
 | > log_func:np.log10
 | > min_level_db:0
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:None
 | > fft_size:1024
 | > power:None
 | > preemphasis:0.0
 | > griffin_lim_iters:None
 | > signal_norm:None
 | > symmetric_norm:None
 | > mel_fmin:0
 | > mel_fmax:None
 | > pitch_fmin:None
 | > pitch_fmax:None
 | > spec_gain:20.0
 | > stft_pad_mode:reflect
 | > max_norm:1.0
 | > clip_norm:True
 | > do_trim_silence:False
 | > trim_db:60
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:None
 | > base:10
 | > hop_length:256
 | > win_length:1024
 | > Found 1501 files in /root/Documents/Audio-Encoder-Pretraining-main/voice_coder/TTS_Training/hindi_temp/hindi


In [18]:
language_manager = LanguageManager(config=config)
config.model_args.num_languages = 11
# language_manager.load_ids_from_file('model_config/language_ids.json')
tokenizer, config = TTSTokenizer.init_from_config(config)

vocab is  ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
final Vocab:  ['<PAD>', '<EOS>', '<BOS>', '<BLNK>', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '!', "'", '(', ')', ',', '-', '.', ':', ';', '?', ' ']
vocab is  ['ऀ', 'ँ', 'ं', 'ः', 'ऄ', 'अ', 'आ', 'इ', 'ई', 'उ', 'ऊ', 'ऋ', 'ऌ', 'ऍ', 'ऎ', 'ए', 'ऐ', 'ऑ', 'ऒ', 'ओ', 'औ', 'क', 'ख', 'ग', 'घ', 'ङ', 'च', 'छ', 'ज', 'झ', 'ञ', 'ट', 'ठ', 'ड', 'ढ', 'ण', 'त', 'थ', 'द', 'ध', 'न', 'ऩ', 'प', 'फ', 'ब', 'भ', 'म', 'य', 'र', 'ऱ', 'ल', 'ळ', 'ऴ', 'व', 'श', 'ष', 'स', 'ह', 'ऺ', 'ऻ', '़', 'ऽ', 'ा', 'ि', 'ी', 'ु', 'ू', 'ृ', 'ॄ', 'ॅ', '

In [19]:
model = Vits(config, ap, tokenizer, speaker_manager, language_manager)
# cp = torch.load('model_config/checkpoint_1433000.pth', 
#                 map_location=torch.device('cuda'))
# model_weights = cp['model'].copy()
# for key in list(model_weights.keys()):
#   if "speaker_encoder" in key:
#     del model_weights[key]
# model.load_state_dict(model_weights)


Number of speakers:  50
 > initialization of speaker-embedding layers.
 > initialization of language-embedding layers.


In [20]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print("Number of parameters ", count_parameters(model))

Number of parameters  115227645


In [ ]:
trainer = Trainer(
    TrainerArgs(), config, output_path, model=model, train_samples=train_samples, eval_samples=eval_samples
)

trainer.fit()

2023-06-15 10:20:27,165.165 DEBUG local:  open file: /root/Documents/Audio-Encoder-Pretraining-main/voice_coder/TTS_Training/./hindi_female-June-15-2023_10+20AM-f6af544/config.json
2023-06-15 10:20:27,262.262 DEBUG local:  open file: /root/Documents/Audio-Encoder-Pretraining-main/voice_coder/TTS_Training/./hindi_female-June-15-2023_10+20AM-f6af544/speakers.pth
2023-06-15 10:20:27,265.265 DEBUG local:  open file: /root/Documents/Audio-Encoder-Pretraining-main/voice_coder/TTS_Training/./hindi_female-June-15-2023_10+20AM-f6af544/language_ids.json


 > `speakers.pth` is saved to ./hindi_female-June-15-2023_10+20AM-f6af544/speakers.pth.
 > `speakers_file` is updated in the config.json.
 > `language_ids.json` is saved to ./hindi_female-June-15-2023_10+20AM-f6af544/language_ids.json.
 > `language_ids_file` is updated in the config.json.


> DataLoader initialization
| > Tokenizer:
	| > add_blank: True
	| > use_eos_bos: False
	| > use_phonemes: False
| > Number of instances : 1486
 | > Preprocessing samples
 | > Max text length: 144
 | > Min text length: 16
 | > Avg text length: 77.88155339805826
 | 
 | > Max audio length: 159914.0
 | > Min audio length: 24078.0
 | > Avg audio length: 104600.20194174757
 | > Num. instances discarded samples: 456
 | > Batch group size: 0.
 [!] Character ' ' not found in the vocabulary. Discarding it. [!] Character ' ' not found in the vocabulary. Discarding it. [!] Character ' ' not found in the vocabulary. Discarding it.


 [!] Character ' ' not found in the vocabulary. Discarding it.
 [!] Character '

/root/Documents/Audio-Encoder-Pretraining-main/voice_coder/TTS_Training/TTS/tts/models/vits.py:1478: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matricesor `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at  /opt/conda/conda-bld/pytorch_1659484808560/work/aten/src/ATen/native/TensorShape.cpp:2981.)
  test_figures["{}-alignment".format(idx)] = plot_alignment(alignment.T, output_fig=False)
2023-06-15 10:20:43,844.844 DEBUG local:  open file: /root/Documents/Audio-Encoder-Pretraining-main/voice_coder/TTS_Training/./hindi_female-June-15-2023_10+20AM-f6af544/best_model_22.pth




> DataLoader initialization
| > Tokenizer:
	| > add_blank: True
	| > use_eos_bos: False
	| > use_phonemes: False
	| > 1 not found characters:
	| >  
| > Number of instances : 1486
 | > Preprocessing samples
 | > Max text length: 144
 | > Min text length: 16
 | > Avg text length: 77.88155339805826
 | 
 | > Max audio length: 159914.0
 | > Min audio length: 24078.0
 | > Avg audio length: 104600.20194174757
 | > Num. instances discarded samples: 456
 | > Batch group size: 0.
 [!] Character '\xa0' not found in the vocabulary. Discarding it.


> DataLoader initialization
| > Tokenizer:
	| > add_blank: True
	| > use_eos_bos: False
	| > use_phonemes: False
	| > 1 not found characters:
	| >  
| > Number of instances : 15
 | > Preprocessing samples
 | > Max text length: 116
 | > Min text length: 52
 | > Avg text length: 79.16666666666667
 | 
 | > Max audio length: 155270.0
 | > Min audio length: 64175.0
 | > Avg audio length: 98906.16666666667
 | > Num. instances discarded samples: 3
 | > Batc

2023-06-15 10:21:04,856.856 DEBUG local:  open file: /root/Documents/Audio-Encoder-Pretraining-main/voice_coder/TTS_Training/./hindi_female-June-15-2023_10+20AM-f6af544/best_model_44.pth


...



> DataLoader initialization
| > Tokenizer:
	| > add_blank: True
	| > use_eos_bos: False
	| > use_phonemes: False
	| > 1 not found characters:
	| >  
| > Number of instances : 1486
 | > Preprocessing samples
 | > Max text length: 144
 | > Min text length: 16
 | > Avg text length: 77.88155339805826
 | 
 | > Max audio length: 159914.0
 | > Min audio length: 24078.0
 | > Avg audio length: 104600.20194174757
 | > Num. instances discarded samples: 456
 | > Batch group size: 0.
 [!] Character '\xa0' not found in the vocabulary. Discarding it.


> DataLoader initialization
| > Tokenizer:
	| > add_blank: True
	| > use_eos_bos: False
	| > use_phonemes: False
	| > 1 not found characters:
	| >  
| > Number of instances : 15
 | > Preprocessing samples
 | > Max text length: 116
 | > Min text length: 52
 | > Avg text length: 79.16666666666667
 | 
 | > Max audio length: 155270.0
 | > Min audio length: 64175.0
 | > Avg audio length: 98906.16666666667
 | > Num. instances discarded samples: 3
 | > Batc

2023-06-15 10:21:44,294.294 DEBUG local:  open file: /root/Documents/Audio-Encoder-Pretraining-main/voice_coder/TTS_Training/./hindi_female-June-15-2023_10+20AM-f6af544/best_model_88.pth




> DataLoader initialization
| > Tokenizer:
	| > add_blank: True
	| > use_eos_bos: False
	| > use_phonemes: False
	| > 1 not found characters:
	| >  
| > Number of instances : 1486
 | > Preprocessing samples
 | > Max text length: 144
 | > Min text length: 16
 | > Avg text length: 77.88155339805826
 | 
 | > Max audio length: 159914.0
 | > Min audio length: 24078.0
 | > Avg audio length: 104600.20194174757
 | > Num. instances discarded samples: 456
 | > Batch group size: 0.
 [!] Character '\xa0' not found in the vocabulary. Discarding it.


2023-06-15 10:21:55,681.681 DEBUG local:  open file: /root/Documents/Audio-Encoder-Pretraining-main/voice_coder/TTS_Training/./hindi_female-June-15-2023_10+20AM-f6af544/checkpoint_100.pth




> DataLoader initialization
| > Tokenizer:
	| > add_blank: True
	| > use_eos_bos: False
	| > use_phonemes: False
	| > 1 not found characters:
	| >  
| > Number of instances : 15
 | > Preprocessing samples
 | > Max text length: 116
 | > Min text length: 52
 | > Avg text length: 79.16666666666667
 | 
 | > Max audio length: 155270.0
 | > Min audio length: 64175.0
 | > Avg audio length: 98906.16666666667
 | > Num. instances discarded samples: 3
 | > Batch group size: 0.
 | > Synthesizing test sentences.
torch.Size([1, 1, 68]) torch.Size([1, 1, 68]) 1.0


> DataLoader initialization
| > Tokenizer:
	| > add_blank: True
	| > use_eos_bos: False
	| > use_phonemes: False
	| > 1 not found characters:
	| >  
| > Number of instances : 1486
 | > Preprocessing samples
 | > Max text length: 144
 | > Min text length: 16
 | > Avg text length: 77.88155339805826
 | 
 | > Max audio length: 159914.0
 | > Min audio length: 24078.0
 | > Avg audio length: 104600.20194174757
 | > Num. instances discarded samp